# HSV Analysis

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot  as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from matplotlib.colors import hsv_to_rgb

In [ ]:
impath = r'F:\TUhh\Thesis\images\Trial2\label\valid\Image_273.jpg'
img = cv2.imread(impath)

In [ ]:
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

In [ ]:
r, g, b = cv2.split(img)
fig = plt.figure()
axis = fig.add_subplot(1, 1, 1, projection="3d")
pixel_colors = img.reshape((np.shape(img)[0]*np.shape(img)[1], 3))
norm = colors.Normalize(vmin=-1.,vmax=1.)
norm.autoscale(pixel_colors)
pixel_colors = norm(pixel_colors).tolist()
axis.scatter(r.flatten(), g.flatten(), b.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Red")
axis.set_ylabel("Green")
axis.set_zlabel("Blue")
plt.show()

In [ ]:
pixel_colors = img.reshape((np.shape(img)[0]*np.shape(img)[1], 3))
norm = colors.Normalize(vmin=-1.,vmax=1.)
norm.autoscale(pixel_colors)
pixel_colors = norm(pixel_colors).tolist()

In [ ]:
axis.scatter(r.flatten(), g.flatten(), b.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Red")
axis.set_ylabel("Green")
axis.set_zlabel("Blue")
plt.show()

In [ ]:
hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

In [ ]:
h, s, v = cv2.split(hsv)
fig = plt.figure()
axis = fig.add_subplot(1, 1, 1, projection="3d")
axis.scatter(h.flatten(), s.flatten(), v.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Hue")
axis.set_ylabel("Saturation")
axis.set_zlabel("Value")
plt.show()

In [ ]:
plt.figure()

axis = plt.subplot(211)
axis.scatter(h.flatten(), v.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Hue")
axis.set_ylabel("Value")

axis1 = plt.subplot(212)
axis1.scatter(s.flatten(), v.flatten(), facecolors=pixel_colors, marker=".")
axis1.set_xlabel("Saturation")
axis1.set_ylabel("Value")

plt.show()

# Pixel Desnity Segmentation

In [ ]:
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cfg = r'F:\TUhh\Thesis\YOLO Colab Files\416_tiny_trial3\yolov4-tiny-custom.cfg'
weight = r'F:\TUhh\Thesis\YOLO Colab Files\416_tiny_trial3\yolov4-tiny-custom_best.weights'
net = cv2.dnn.readNetFromDarknet(cfg, weight)
classes = ['Corrosion']

In [ ]:
#impath = r'F:\TUhh\Thesis\images\Trial2\label\valid\Image_184.jpg'
impath = r'F:\TUhh\Thesis\images\sample1_5.JPG'
img = cv2.imread(impath)
img = cv2.resize(img,(416,416))
hight,width,_ = img.shape
det=1

Img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
hsv = cv2.cvtColor(Img, cv2.COLOR_RGB2HSV)

m = Img.shape[0]
n = Img.shape[1]
template = np.ones((m,n))
template = template.astype(int)

rn = template
gn = template
bn = template


blob = cv2.dnn.blobFromImage(img, 1/255,(416,416),(0,0,0),swapRB = True,crop= False)

net.setInput(blob)

output_layers_name = net.getUnconnectedOutLayersNames()

layerOutputs = net.forward(output_layers_name)

boxes =[]
confidences = []
class_ids = []

for output in layerOutputs:
    for detection in output:
        score = detection[5:]
        class_id = np.argmax(score)
        confidence = score[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * hight)
            w = int(detection[2] * width)
            h = int(detection[3]* hight)
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            boxes.append([x,y,w,h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes,confidences,.5,.4)

font = cv2.FONT_HERSHEY_PLAIN

if  len(indexes)>0:
    for i in indexes.flatten():
        x,y,w,h = boxes[i]
        for x1 in range(y,y+h):
            for y1 in range(x,x+w):
                gn[x1][y1] = 255
        label = str(classes[class_ids[i]])
        confidence = str(round(confidences[i],2))
        color = (0,0,255)
        cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
        cv2.putText(img,str(det), (x,y),font,2,color,2)
        det=det+1
                    
    skin = np.dstack([rn, gn, bn])
    
    blight_mask = (0, 0, 0)
    bdark_mask = (20, 180, 120)
    rlight_mask = (0, 125, 0)
    rdark_mask = (20, 180,180)
    mask = cv2.inRange(hsv, blight_mask, bdark_mask)
    rmask = cv2.inRange(hsv, rlight_mask, rdark_mask)
    m = cv2.bitwise_not(mask)
    rm = cv2.bitwise_not(rmask)
    skinPatch = cv2.bitwise_and(skin,skin,mask = mask)
    skinPatch1 = cv2.bitwise_and(skin,skin,mask = rmask) 
    newSkin = cv2.bitwise_or(skinPatch,skinPatch1)
    newSkin = newSkin.astype(np.uint8)
    gry = cv2.cvtColor(newSkin, cv2.COLOR_RGB2GRAY)
    plt.imshow(gry)
    cv2.imwrite(r'F:\TUhh\Thesis\Documentation\pixDensresult7.jpg', gry)
    
            
    for i in indexes.flatten():
        x,y,w,h = boxes[i]
        ctr = 0
        for x1 in range (y,y+h):
            for y1 in range(x,x+w):
                if gry[x1][y1]==255:
                    ctr = ctr+1
        print("number of pixels:"+ str(ctr))

            


cv2.imwrite(r'F:\TUhh\Thesis\Documentation\pixDensresult8.jpg', img)
cv2.waitKey(0) 
cv2.destroyAllWindows()